In [1]:
import torch
torch.cuda.is_available()
print(torch.zeros(1).cuda())

import torch_geometric
import torch_geometric.nn as pyg_nn
from torch_sparse import SparseTensor, matmul
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
print(torch_geometric.__version__)

import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType, OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

import networkx as nx
from deepsnap.hetero_graph import HeteroGraph
import copy
from copy import deepcopy
import pickle

import deepsnap
from deepsnap.graph import Graph
from deepsnap.batch import Batch
from deepsnap.dataset import GraphDataset
from deepsnap.hetero_gnn import forward_op
from deepsnap.hetero_graph import HeteroGraph
from sklearn.metrics import f1_score, roc_auc_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
from pathlib import Path as Data_Path
from os import listdir
from os.path import isfile, join
from itertools import combinations
from tqdm.notebook import tqdm

import graph_tool.all as gt
import json
print("graph-tool version: {}".format(gt.__version__.split(' ')[0]))

/home/asd27/.conda/envs/cudatorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([0.], device='cuda:0')
2.1.0
graph-tool version: 2.46


In [2]:
import pickle

with open("final_graph_pta.pickle", "rb") as f:
    final_graph = pickle.load(f)

In [3]:
count_pl = sum(1 for v in final_graph.vertices() if final_graph.vp.pl_type[v] == "playlist")
print(f'num_playlists: {count_pl}')
count_tr = sum(1 for v in final_graph.vertices() if final_graph.vp.tr_type[v] == "track")
print(f'num_tracks: {count_tr}')
count_ar = sum(1 for v in final_graph.vertices() if final_graph.vp.ar_type[v] == "artist")
print(f'num_artist: {count_ar}')

num_playlists: 76000
num_tracks: 2662703
num_artist: 753005


In [4]:
labels, hist = gt.label_components(final_graph)

# Get the label of the largest component
largest_label = labels[np.argmax(hist)]

# Extract the largest component as a subgraph
largest_component = gt.GraphView(final_graph, vfilt=labels.a == largest_label)

print("Number of vertices:", largest_component.num_vertices())
print("Number of edges:", largest_component.num_edges())

Number of vertices: 49100
Number of edges: 5822388


In [5]:
#Remove artist nodes in largest component

vp = largest_component.vertex_properties["ar_type"]
v_list = [v for v in largest_component.vertices() if vp[v] == "artist"]

for v in v_list:
    largest_component.clear_vertex(v)

In [6]:
count_pl = sum(1 for v in largest_component.vertices() if largest_component.vp.pl_type[v] == "playlist")
print(f'num_playlists: {count_pl}')
count_tr = sum(1 for v in largest_component.vertices() if largest_component.vp.tr_type[v] == "track")
print(f'num_tracks: {count_tr}')
count_ar = sum(1 for v in largest_component.vertices() if largest_component.vp.ar_type[v] == "artist")
print(f'num_artist: {count_ar}')

num_playlists: 2000
num_tracks: 37256
num_artist: 9844


In [7]:
print("Number of vertices:", largest_component.num_vertices())
print("Number of edges:", largest_component.num_edges())

Number of vertices: 49100
Number of edges: 5103910
